In [17]:

from pyspark.sql.functions import sum, when, col, lit, min, max, count, countDistinct, isnan, greatest
from pyspark.sql.functions import row_number, regexp_replace, regexp_extract, substring, concat, replace, split, explode, map_from_arrays, regexp_extract_all, coalesce, position, posexplode, expr
from pyspark.sql.functions import dateadd, format_number
from pyspark.sql.types import *
from pyspark.sql.window import Window
from pyspark.sql import SparkSession
import pandas as pd
from functools import reduce
from pyspark.sql import functions as F

ImportError: cannot import name 'map_agg' from 'pyspark.sql.functions' (c:\Users\kerle\miniconda3\envs\pyspark\lib\site-packages\pyspark\sql\functions.py)

In [3]:
spark = SparkSession \
    .builder \
    .appName("App Ad").getOrCreate()
#    .config("spark.some.config.option", "some-value") \

spark.conf.set("spark.sql.repl.eagerEval.enabled", True)

In [4]:
file = "data/3.csv"

df = spark.read.format("csv") \
       .option("sep", ";")\
       .option("header", "false") \
       .option("inferSchema", "true") \
       .load(file) 

column_names = []
for i in range(len(df.columns)):
    column_names.append("set" + str(i))
df = df.toDF(*column_names)

In [5]:
print(df.count())
print(len(df.columns))


140
1


In [6]:
df.show(truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------+
|set0                                                                                                                                        |
+--------------------------------------------------------------------------------------------------------------------------------------------+
|...317..........214.....................................751.................................630...479..205....41.993............416.........|
|...*....813........%....572........%...629.154......518....*....365..................-.......*.......#.....................422...........661|
|269.......*...58...........=......264.....*..........*......937.-...........235...303.........848..............195.....154*.........144.-...|
|........476..@...162.855................$....288...821..............107.....-...........290......../..301.........=...........135..*........|

In [48]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, split, posexplode, expr, collect_list, struct

df = df.withColumn("all_numbers", split(col("set0"), "\\D+"))
df = df.withColumn("all_symbols", split(col("set0"), "[.\d+]+"))

# Explode the array of matches with their positions
exploded_df = df.select("set0", posexplode(col("all_numbers")).alias("num_pos", "num_match"), posexplode(col("all_symbols")).alias("sym_pos", "sym_match"))
# exploded_df = exploded_df.withColumns('("set0", posexplode(col("all_numbers")).alias("num_pos", "num_match"), posexplode(col("all_symbols")).alias("sym_pos", "sym_match"))

# Filter out empty matches (from consecutive non-digit characters)
filtered_df = exploded_df.filter((col("num_match") != "") | (col('sym_match_') != ""))

# Calculate the actual position of each match in the original string
result_df = filtered_df.withColumn("actual_num_pos", expr("instr(set0, num_match)"))\
                        .withColumn('actual_sym_pos', expr('instr(set0, sym_match)'))

# Group by original text and collect matches and positions into a list of structs
final_df = result_df.groupBy("set0").agg(collect_list(struct("num_match", "actual_num_pos")).alias("match_num_positions"),
                                         collect_list(struct("sym_match", "actual_sym_pos")).alias("match_sym_positions"))




AnalysisException: [UNSUPPORTED_GENERATOR.MULTI_GENERATOR] The generator is not supported: only one generator allowed per SELECT clause but found 2: "posexplode(all_numbers)", "posexplode(all_symbols)".

In [44]:
df.show(truncate=False)
# final_df.show(truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------+----------------------------------------------------------------------------------+-------------------------------+
|set0                                                                                                                                        |all_matches                                                                       |all_numbers                                                                       |all_symbols                    |
+--------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------+----------------------------------------------------------------------------------+----------

In [10]:
# Extract the first digit
df = df.withColumn("first_digit", regexp_extract_all("3", col("set0"), 0))

# Find the position of the first digit
df = df.withColumn("digit_position", position(col("first_digit"), col("set0")))

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `3` cannot be resolved. Did you mean one of the following? [`set0`].;
'Project [set0#19, regexp_extract_all('3, set0#19, 0) AS first_digit#48]
+- Project [_c0#17 AS set0#19]
   +- Relation [_c0#17] csv
